In [1]:
import pickle
import numpy as np
import pandas as pd

DATA = {
    "X" : [],
    "category" : [],
    "y" : [],
    "topic":[]
}

X=DATA["X"]
category=DATA["category"]
y=DATA["y"]
topic = DATA["topic"]

with open('covid_fake_news_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(0,len(dst)))
    data = pd.read_csv("raw/covid_fake_news.csv",sep="\t")
    y.extend(1-data["label"].values)
    
    with open('covid_fake_news_categories.pickle', 'rb') as handle:
        top=pickle.load(handle)
        topic.extend(top)
    
with open('mmcovid_en_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(1,len(dst)))
    data = pd.read_csv("raw/mmcovid_en.csv",sep=",")
    y.extend(data["label"].values)
    

with open('pubhealth_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(2,len(dst)))
    data = pd.read_csv("raw/pubhealth.csv",sep=",")
    y.extend(data["label"].values)

    
with open('qprop_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(3,len(dst)))
    data = pd.read_csv("raw/qprop.csv",sep="\t")
    y.extend(data["label"].values)
    
with open('isot_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(4,len(dst)))
    data = pd.read_csv("raw/isot.csv",sep=",")
    y.extend(data["label"].values)
    
with open('grafn_BERTEmbeddings.pickle', 'rb') as handle:
    dst = pickle.load(handle)
    X.extend(dst)
    category.extend(np.repeat(5,len(dst)))
    data = pd.read_csv("raw/grafn.csv",sep=",")
    y.extend(data["label"].values)
    
    
DATA["X"]=np.array(X)
DATA["category"]=np.array(category)
DATA["y"]=np.array(y)
DATA["folds"] = []
DATA["topic"] = np.array(topic)

/tmp/ipykernel_3903/8608421.py:62: DtypeWarning: Columns (2,4,5,8,9,10,11,12,13,15,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("raw/grafn.csv",sep=",")


In [2]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def experiment(foldids, X, y, cls = LogisticRegression(max_iter=10000), fit=True):

    scores = {
        'Accuracy': {'func': accuracy_score},
        'Balanced Accuracy': {'func': balanced_accuracy_score},
        'F1': {'func': f1_score},
        'Precision': {'func': precision_score},
        'Recall': {'func': recall_score},
        'G-mean': {'func': geometric_mean_score}
    }

    for score_name, score_dict in scores.items():
        scores[score_name]["list"] = []
        scores[score_name]["lab"] = []

    for fold,j in enumerate(foldids):
        train = foldids[fold][1]
        test = foldids[fold][2]
        xin, yin = X[train], np.array(y[train])
        
        pca = PCA(n_components=128)
        pca.fit(xin)
        
        
        if fit == True:
            cls.fit(pca.transform(xin), yin)
        y_pred = cls.predict(pca.transform(X[test]))
        for score_name, score_dict in scores.items():
            if score_name in ["F1","Precision","Recall"]:
                scorvaln = score_dict['func'](y[test], y_pred, average=None)
                score_dict['lab'].append(scorvaln)
                scorval = score_dict['func'](y[test], y_pred, average="weighted")
                score_dict['list'].append(scorval)
                #print(score_name, scorval, scorvaln)  
            else:
                scorval=score_dict['func'](y[test], y_pred)
                score_dict['list'].append(scorval)
                #print(score_name, scorval)
        #print(" ")

    #clear_output()
    for score_name, score_dict in scores.items():
        score_dict['avg'] = np.mean(score_dict['list'])
        score_dict['std'] = np.std(score_dict['list'])
 
    # Print stats
    numlabels = scores["F1"]["lab"][0].shape[0]
    scores["F1"]["lab"][0].shape[0] 
    head = "| %-20s | %-10s |" +  numlabels * " %-10s |" 
    headv = ["Score", "Average"]
    headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
    row=head % tuple(headv)
    print("+"*len(row))
    print(row)
    print("+"*len(row))
    for score_name, score_dict in sorted(scores.items()) :
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
        for i in range(numlabels):
            if score_name in ["F1","Precision","Recall"]:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels* " %4.1f ± %3.1f |"
                vals = [v[i] for v in scores[score_name]["lab"]]
                headv.append(np.mean(vals)*100)
                headv.append(np.std(vals)*100)
            else:
                head = "| %-20s | %4.1f ± %3.1f |" + numlabels * " %-10s |" 
                headv.append("-")
        print(head % tuple(headv))
    print("+"*len(row))
    return cls, scores, pca

<hr style="border: 5px dashed red">

# Topic aware

In [3]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.decomposition import PCA

t=0
X = DATA["X"][DATA["category"] == t]
y = DATA["y"][DATA["category"] == t]
topics = DATA["topic"]
foldids = []

print(X.shape, y.shape, topics.shape)

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)

for fold_idx, (train, test) in enumerate(rskf.split(X, y)):
    foldids.append((fold_idx,train,test))
    
for _foldid in range(10):
    print("fold",_foldid) 
    trainX = X[foldids[_foldid][1]]
    trainY = y[foldids[_foldid][1]]
    
    testX = X[foldids[_foldid][2]]
    testY = y[foldids[_foldid][2]]
    
    pca = PCA(n_components=128)
    pca.fit(trainX)
    
    lr = LogisticRegression(max_iter=10000, class_weight='balanced')
    lr.fit(pca.transform(trainX), trainY)
    
    scr = balanced_accuracy_score(testY, lr.predict(pca.transform(testX)))
    print(scr)

(8972, 768) (8972,) (8972,)
fold 0
0.8922544132069303
fold 1
0.8945925598099511
fold 2
0.8896095537757438
fold 3
0.8979321501060631
fold 4
0.8893745913697286
fold 5
0.8847594630203326
fold 6
0.9017714122262177
fold 7
0.892075531205966
fold 8
0.8769185191238966
fold 9
0.8999603216994521


In [4]:

for _foldid in range(10):
    print("fold",_foldid)
    train_topics = topics[foldids[_foldid][1]]
    test_topics = topics[foldids[_foldid][2]]
    
    trainX = X[foldids[_foldid][1]]
    trainY = y[foldids[_foldid][1]]
    
    testX = X[foldids[_foldid][2]]
    testY = y[foldids[_foldid][2]]
    
    pca = PCA(n_components=128)
    pca.fit(trainX)

    cls = {}
    #print(np.unique(train_topics,return_counts=True))
    for topic in np.unique(train_topics):
        cls[topic] = LogisticRegression(max_iter=10000, class_weight='balanced')
        this_topic = train_topics==topic
        if len(np.unique(trainY[this_topic])) > 1:
            cls[topic].fit(pca.transform(trainX[this_topic]), trainY[this_topic])
        else:
            #print(trainY[this_topic])
            cls[topic] = np.max(trainY[this_topic])
    
    pred = []
    gt = []
    #print(np.unique(test_topics,return_counts=True))
    for topic in np.unique(test_topics):
        this_topic = test_topics==topic
        gt.extend(testY[this_topic])
        
        if topic in cls:
            #print(topic, len(testX[this_topic]))
            if isinstance( cls[topic], LogisticRegression):
                pr = cls[topic].predict(pca.transform(testX[this_topic]))
                pred.extend(pr)
            else:
                pred.extend(np.repeat(cls[topic],len(testX[this_topic])))
        else:
            print("uknown [",topic,"] len=",len(testX[this_topic]), testY[this_topic])
            pred.extend(np.repeat(1,len(testX[this_topic])))
    #test = foldids[2]
    scr = balanced_accuracy_score(gt, pred)
    print(scr)
    

fold 0
0.8332022311212814
fold 1
uknown [ Finance ] len= 1 [1]
0.8207222468092034
fold 2
uknown [ Finance ] len= 1 [1]
0.833025498528931
fold 3
0.803288720680025
fold 4
0.8068210608041844
fold 5
uknown [ Finance ] len= 1 [1]
uknown [ Opinion, Recommendation, Review ] len= 2 [1 1]
0.845471841124015
fold 6
uknown [ Finance ] len= 1 [1]
uknown [ Opinion, Recommendation, Review ] len= 2 [1 1]
0.8213917947041517
fold 7
0.8241111806329198
fold 8
uknown [ Finance ] len= 1 [1]
0.785727566198104
fold 9
0.8494976625411408
